In [2]:
import requests
import pandas as pd
import json
from pprint import pprint
from bs4 import BeautifulSoup
import urllib.parse
import numpy as np

In [ ]:
urls = []
base = "https://www.skiresort.info/ski-resorts/africa"

            

url = base
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
resorts_div = soup.findAll('a', class_='h3')
for elm in resorts_div:
    urls.append(elm.get('href'))

In [3]:
links = ["https://www.skiresort.info/ski-resort/cerro-bayo/", "https://www.skiresort.info/ski-resort/los-penitentes/", "https://www.skiresort.info/ski-resort/la-hoya/",
            "https://www.skiresort.info/ski-resort/cerro-perito-moreno-el-bolson-laderas/", "https://www.skiresort.info/ski-resort/vallecitos/", "https://www.skiresort.info/ski-resort/los-puquios/", "https://www.skiresort.info/ski-resort/glaciar-martial/", 
            "https://www.skiresort.info/ski-resort/batea-mahuida/", "https://www.skiresort.info/ski-resort/valle-hermoso/", "https://www.skiresort.info/ski-resort/antuco-ski-center/"]
                
                
                
                
                

In [ ]:
urls

In [ ]:
urls[:] = [x for x in urls if "report" not in x]
urls[:] = [x for x in urls if "result" not in x]
urls[:] = [x for x in urls if "webcams" not in x]
urls[:] = [x for x in urls if "photos" not in x]
urls[:] = [x for x in urls if "trail-map" not in x]
urls[:] = [x for x in urls if "arosa" not in x]
print(len(urls))

In [ ]:
urls

In [4]:
resort_dictionary = {}
resort_data = []
test_urls = ['https://www.skiresort.info/ski-resort/sky-mountain-beidahu/', 'https://www.skiresort.info/ski-resort/crested-butte/',  'https://www.skiresort.info/ski-resort/erciyes-kayseri/','https://www.skiresort.info/ski-resort/myoko-suginohara/', 'https://www.skiresort.info/ski-resort/kiroro/',  'https://www.skiresort.info/ski-resort/hachikogenhachi-kita-kogen/','https://www.skiresort.info/ski-resort/aomori-spring/', 'https://www.skiresort.info/ski-resort/niki-chomin/','https://www.skiresort.info/ski-resort/muroran-kogen-danpara/','https://www.skiresort.info/ski-resort/yunotani-yakushi/','https://www.skiresort.info/ski-resort/ushidake-onsen/','https://www.skiresort.info/ski-resort/akan-royal-valley/','https://www.skiresort.info/ski-resort/akitaken-tazawako/']
failed_links = []

for url in links:
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        s = soup.find('div', class_='panel-simple more-padding')
        im = soup.find('div', class_='panel-image')

        # basic data -----------------------------------------------
        name = s.find('p', class_='bodytext').get_text()
        description = s.findAll('p')[1].get_text()

        # elevation -----------------------------------------------
        alt = s.find('div', class_='description').get_text()
        elevation_base = alt.split(' m', 1)[0]
        elevation_testing = alt.split('m - ', 1)[1]
        elevation_summit = elevation_testing.split(' m ', 1)[0]

        # images -----------------------------------------------
        try:
            img_url = im.find('a').get('href')
            full_img_url = f'https://www.skiresort.info/{img_url}'
        except:
            full_img_url="n/a"

        r2 = requests.get(f"{url}/trail-map/")
        soup2 = BeautifulSoup(r2.text, 'html.parser')
        try:
            map_url = soup2.find('div', class_='mobile-trailmap-image').find('a').get('href')
            full_map_url = f'https://www.skiresort.info/{map_url}'
        except:
            full_map_url="n/a"


        # runs -----------------------------------------------
        runs = str(s.findAll('div', class_='description')[1])
        runs_total = str(s.findAll('div', class_='description')[1].find(id='selSlopetot').get_text())
        runs_testing = runs_total.split('al: ', 1)[1]
        runs_total2=runs_testing.split(' k', 1)[0]
        try:
            runs_green_testing = runs.split('ner">', 1)[1]
            runs_green = runs_green_testing.split(' km', 1)[0]
        except:
            runs_green=0
        try:
            runs_blue_testing = runs.split('Inter">', 1)[1]
            runs_blue = runs_blue_testing.split(' km', 1)[0]
        except:
            runs_blue=0

        try:
            runs_black_testing = runs.split('Adv">', 1)[1]
            runs_black = runs_black_testing.split(' km', 1)[0]
        except:
            runs_black=0



        # lat & lng -----------------------------------------------
        r3 = requests.get(f"{url}hotels-accommodations/")
        soup3 = BeautifulSoup(r3.text, 'html.parser')
        latlng = soup3.find('div', class_='search-hotel').find('div', class_='stay22trivago')
        latlng_link = latlng.findAll('optgroup')[1].find('option').get('value')
        try:
            lat_testing = latlng_link.split('lat=', 1)[1]
            lat2 = str(lat_testing.split('&', 1)[0])
            lng_testing = latlng_link.split('lng=', 1)[1]
            lng2 = str(lng_testing.split('&', 1)[0])
        except:
            print("error")

        # lifts -----------------------------------------------
        lifts = str(s.findAll('div', class_='lift-count'))
        lifts_total = str(s.find('div', class_='lift-info-group').find(id='selLiftstot'))
        lifts_testing = lifts_total.split('al: ', 1)[1]
        lifts_total2=lifts_testing.split('<', 1)[0]
        try:
            lifts_by_1 = s.find('div', attrs={'title':'Chairlift'})
            chairlifts = int(lifts_by_1.find('span', class_='lift-amount').get_text())
        except:
            chairlifts=0

        try:
            lifts_by_2 = s.find('div', attrs={'title':'Circulating ropeway/gondola lift'})
            gondola = int(lifts_by_2.find('span', class_='lift-amount').get_text())
        except:
            gondolas=0

        try:
            lifts_by_3 = s.find('div', attrs={'title':'Aerial tramway/reversible ropeway'})
            trams = int(lifts_by_3.find('span', class_='lift-amount').get_text())
        except:
            trams=0

        try:
            lifts_by_4 = s.find('div', attrs={'title':'Combined installation (gondola and chair)'})
            combo = int(lifts_by_4.find('span', class_='lift-amount').get_text())
        except:
            combo=0

        try:
            lifts_by_5 = s.find('div', attrs={'title':'T-bar lift/platter/button lift '})
            surface = int(lifts_by_5.find('span', class_='lift-amount').get_text())
            print(surface)
        except:
            surface=0

        try:
            lifts_by_6 = s.find('div', attrs={'title':'Rope tow/baby lift'})
            ropetow = int(lifts_by_6.find('span', class_='lift-amount').get_text())
            print(ropetow)
        except:
            ropetow=0

        try:
            lifts_by_7 = s.find('div', attrs={'title':'People mover'})
            pplmover = int(lifts_by_7.find('span', class_='lift-amount').get_text())
        except:
            pplmover=0

        try:
            lifts_by_8 = s.find('div', attrs={'title':'Funicular'})
            funicular = lifts_by_8.find('span', class_='lift-amount').get_text()
        except:
            funicular=0

        try:
            lifts_by_9 = s.find('div', attrs={'title':'Cog railway'})
            railway = int(lifts_by_9.find('span', class_='lift-amount').get_text())
        except:
            railway=0

        gdtr=gondolas+trams+combo
        surface_lifts = surface+ropetow+pplmover
        trains = funicular+railway

        # dictionary creation  -----------------------------------------------
        resort_dictionary = {
                "name":name,
                "image_url": full_img_url,
                "lat":lat2,
                "lng":lng2,
                "description":description,
                "lift_breakdown":{
                    "gondolasAndTrams":gdtr,
                    "trains":trains,
                    "chairlifts":chairlifts,
                    "surface":surface_lifts
                },
                "lifts_total":lifts_total2,
                "runs_total":runs_total2,
                "runsGreen_total":runs_green,
                "runsBlue_total":runs_blue,
                "runsBlack_total":runs_black,
                "elevation_base":elevation_base,
                "elevation_summit":elevation_summit,
                "trailMap_image":full_map_url
            }
        resort_data.append(resort_dictionary)
    except:
        failed_links.append(url)
        print('error')


1
3
error
2
3
5
1
1
3
2
4
5
1


In [5]:
resort_data

[{'name': 'Skiing Los Penitentes',
  'image_url': 'n/a',
  'lat': '-32.840754',
  'lng': '-69.839691',
  'description': 'The ski resort Los Penitentes is located in the Mendoza Province (Argentina). For skiing and snowboarding, there are 25.5 km of slopes available. 7 lifts transport the guests. The winter sports area is situated between the elevations of 2,579 and 3,194 m. ',
  'lift_breakdown': {'gondolasAndTrams': 0,
   'trains': 0,
   'chairlifts': 2,
   'surface': 5},
  'lifts_total': '7',
  'runs_total': '25.5',
  'runsGreen_total': '1.2',
  'runsBlue_total': '10',
  'runsBlack_total': '14.3',
  'elevation_base': ' 2579',
  'elevation_summit': '3194',
  'trailMap_image': 'https://www.skiresort.info/typo3temp/assets/_processed_/dc/c2/25/57/0f6449af40.jpg'},
 {'name': 'Skiing La Hoya',
  'image_url': 'n/a',
  'lat': '-42.824263',
  'lng': '-71.252981',
  'description': 'The ski resort La Hoya is located in the Chubut Province (Argentina). For skiing and snowboarding, there are 14 k

In [6]:
resort_df = pd.DataFrame(resort_data)

In [7]:
resort_df.head(60)

,name,image_url,lat,lng,description,lift_breakdown,lifts_total,runs_total,runsGreen_total,runsBlue_total,runsBlack_total,elevation_base,elevation_summit,trailMap_image
0,Skiing Los Penitentes,n/a,-32.840754,-69.839691,The ski resort Los Penitentes is located in th...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",7,25.5,1.2,10,14.3,2579,3194,https://www.skiresort.info/typo3temp/assets/_p...
1,Skiing La Hoya,n/a,-42.824263,-71.252981,The ski resort La Hoya is located in the Chubu...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",10,14,2.8,5.6,5.6,1430,2075,https://www.skiresort.info/typo3temp/assets/_p...
2,Skiing Cerro Perito Moreno – El Bolsón (Laderas),n/a,-41.791037,-71.564244,The ski resort Cerro Perito Moreno – El Bolsón...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",6,11.5,3,6.5,2,990,1650,https://www.skiresort.info/typo3temp/assets/_p...
3,Skiing Vallecitos,n/a,-32.973836,-69.359198,The ski resort Vallecitos is located in the Me...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",6,4,1,2,1,3000,3650,n/a
4,Skiing Los Puquios,n/a,-32.840758,-69.839654,The ski resort Los Puquios is located in the M...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",4,7,5,2,0,2670,2985,https://www.skiresort.info/typo3temp/assets/_p...
5,Skiing Glaciar Martial,n/a,-54.798607,-68.370716,The ski resort Glaciar Martial is located in t...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",1,5,0.5,3.5,1,330,520,https://www.skiresort.info/typo3temp/assets/_p...
6,Skiing Batea Mahuida,n/a,-38.834117,-71.223309,The ski resort Batea Mahuida is located in the...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",5,2.5,2.5,0,0,1650,1740,https://www.skiresort.info/typo3temp/assets/_p...
7,Skiing Valle Hermoso,n/a,-36.917716,-71.420238,The ski resort Valle Hermoso is located in Chi...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",1,1.7,1.3,0.4,0,1602,1655,https://www.skiresort.info/typo3temp/assets/_p...
8,Skiing Antuco Ski Center,n/a,-37.406825,-71.301313,The ski resort Antuco Ski Center is located in...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",3,9,3,4,2,1400,1850,https://www.skiresort.info/typo3temp/assets/_p...


In [8]:
resort_df['runsBlue_total'].replace('n/a', np.NaN, inplace=True)
resort_df['runsGreen_total'].replace('n/a', np.NaN, inplace=True)
resort_df['runsBlack_total'].replace('n/a', np.NaN, inplace=True)
resort_df['runs_total'].replace('n/a', np.NaN, inplace=True)
resort_df['runs_total'].replace('n/a', np.NaN, inplace=True)

In [9]:
resort_df['runsBlue_total'] = pd.to_numeric(resort_df['runsBlue_total'])
resort_df['runsGreen_total'] = pd.to_numeric(resort_df['runsGreen_total'])
resort_df['runsBlack_total'] = pd.to_numeric(resort_df['runsBlack_total'])
resort_df['runs_total'] = pd.to_numeric(resort_df['runs_total'])

In [10]:
resort_df['elevation_base'] = resort_df['elevation_base'].str.replace('(','', regex=True)

In [11]:
resort_df['elevation_summit'] = pd.to_numeric(resort_df['elevation_summit'])
resort_df['elevation_base'] = pd.to_numeric(resort_df['elevation_base'])

In [12]:
resort_df['total_runs'] = ((resort_df['runsBlue_total'])+(resort_df['runsGreen_total'])+(resort_df['runsBlack_total']))
resort_df['runsBlue_percent'] = (resort_df['runsBlue_total'])/(resort_df['total_runs'])*100
resort_df['runsGreen_percent'] = (resort_df['runsGreen_total'])/(resort_df['total_runs'])*100
resort_df['runsBlack_percent'] = (resort_df['runsBlack_total'])/(resort_df['total_runs'])*100

In [13]:
resort_df['vertical_drop'] =(resort_df['elevation_summit'])-(resort_df['elevation_base'])

In [14]:
resort_df.to_json('resorts_extra.json')

In [15]:
resort_df

,name,image_url,lat,lng,description,lift_breakdown,lifts_total,runs_total,runsGreen_total,runsBlue_total,runsBlack_total,elevation_base,elevation_summit,trailMap_image,total_runs,runsBlue_percent,runsGreen_percent,runsBlack_percent,vertical_drop
0,Skiing Los Penitentes,n/a,-32.840754,-69.839691,The ski resort Los Penitentes is located in th...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",7,25.5,1.2,10.0,14.3,2579,3194,https://www.skiresort.info/typo3temp/assets/_p...,25.5,39.215686,4.705882,56.078431,615
1,Skiing La Hoya,n/a,-42.824263,-71.252981,The ski resort La Hoya is located in the Chubu...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",10,14.0,2.8,5.6,5.6,1430,2075,https://www.skiresort.info/typo3temp/assets/_p...,14.0,40.000000,20.000000,40.000000,645
2,Skiing Cerro Perito Moreno – El Bolsón (Laderas),n/a,-41.791037,-71.564244,The ski resort Cerro Perito Moreno – El Bolsón...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",6,11.5,3.0,6.5,2.0,990,1650,https://www.skiresort.info/typo3temp/assets/_p...,11.5,56.521739,26.086957,17.391304,660
3,Skiing Vallecitos,n/a,-32.973836,-69.359198,The ski resort Vallecitos is located in the Me...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",6,4.0,1.0,2.0,1.0,3000,3650,n/a,4.0,50.000000,25.000000,25.000000,650
4,Skiing Los Puquios,n/a,-32.840758,-69.839654,The ski resort Los Puquios is located in the M...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",4,7.0,5.0,2.0,0.0,2670,2985,https://www.skiresort.info/typo3temp/assets/_p...,7.0,28.571429,71.428571,0.000000,315
5,Skiing Glaciar Martial,n/a,-54.798607,-68.370716,The ski resort Glaciar Martial is located in t...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",1,5.0,0.5,3.5,1.0,330,520,https://www.skiresort.info/typo3temp/assets/_p...,5.0,70.000000,10.000000,20.000000,190
6,Skiing Batea Mahuida,n/a,-38.834117,-71.223309,The ski resort Batea Mahuida is located in the...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",5,2.5,2.5,0.0,0.0,1650,1740,https://www.skiresort.info/typo3temp/assets/_p...,2.5,0.000000,100.000000,0.000000,90
7,Skiing Valle Hermoso,n/a,-36.917716,-71.420238,The ski resort Valle Hermoso is located in Chi...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",1,1.7,1.3,0.4,0.0,1602,1655,https://www.skiresort.info/typo3temp/assets/_p...,1.7,23.529412,76.470588,0.000000,53
8,Skiing Antuco Ski Center,n/a,-37.406825,-71.301313,The ski resort Antuco Ski Center is located in...,"{'gondolasAndTrams': 0, 'trains': 0, 'chairlif...",3,9.0,3.0,4.0,2.0,1400,1850,https://www.skiresort.info/typo3temp/assets/_p...,9.0,44.444444,33.333333,22.222222,450


In [16]:
with open('resorts_extra.json', 'w', encoding='utf-8') as file:
    resort_df.to_json(file, orient='records', indent=2, force_ascii=False)